In [ ]:
!pip install transformers peft evaluate rouge_score -Uqqq

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

2024-04-10 13:52:01.846281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 13:52:01.846503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 13:52:02.095786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import pandas as pd
from peft import get_peft_model, PeftConfig, PeftModel
from transformers import AutoTokenizer,AutoModel, AutoModelForCausalLM, TrainingArguments, GenerationConfig


In [ ]:
#test_ds = pd.read_csv('/kaggle/input/tinycoder/test_dataset_python.csv')

In [ ]:
config = PeftConfig.from_pretrained("SSahas/codegen")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")
model = PeftModel.from_pretrained(model, "SSahas/codegen")
tokenizer = AutoTokenizer.from_pretrained("SSahas/codegen")

adapter_config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


adapter_model.safetensors:   0%|          | 0.00/5.25M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate_code(input_text):

    wrapped_text = f"""Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {input_text}

    ### Input:


    ### Response:
    """

    input_ids = tokenizer(wrapped_text, padding = True, return_tensors = "pt")

    output = model.generate(input_ids=input_ids['input_ids'], generation_config=GenerationConfig(max_new_tokens =  256, eos_token_id = 50256, pad_token_id = 5-256))
    output = tokenizer.decode(output[0][input_ids['input_ids'].shape[1]:], skip_special_tokens=True) #original

    generated_code = output[0:output.find("Output")]

    return generated_code


In [ ]:
input_text = "write a python function to generate fibonacci numbers and return them in a list"
code = generate_code(input_text)
print(code)

def generate_fibonacci_numbers(n):
    fibonacci_numbers = []
    a, b = 0, 1
    while a < n:
        fibonacci_numbers.append(a)
        a, b = b, a + b
    return fibonacci_numbers

print(generate_fibonacci_numbers(10))
# 


In [ ]:
def generate_fibonacci_numbers(n):
    fibonacci_numbers = []
    a, b = 0, 1
    while a < n:
        fibonacci_numbers.append(a)
        a, b = b, a + b
    return fibonacci_numbers

print(generate_fibonacci_numbers(100))


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
